In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import torch
import json
import pandas as pd
import torch.nn as nn
import numpy as np
import torch.optim as optim
from tqdm.auto import tqdm
from torch.utils.data import DataLoader, TensorDataset
from sklearn.metrics import accuracy_score, f1_score
from transformers import BertTokenizer, BertForSequenceClassification, get_scheduler
from transformers import ElectraTokenizer, ElectraForSequenceClassification

In [3]:
import os

# Define the new directory path
new_directory = '/content/drive/MyDrive/Colab Notebooks'

# Change the current working directory to the new one
os.chdir(new_directory)

# Verify the change by printing the current working directory
print("Current working directory:", os.getcwd())

Current working directory: /content/drive/MyDrive/Colab Notebooks


In [4]:
# Load the datasets
def load_dataset(file_path):
    with open(file_path, 'r') as file:
        data = [json.loads(line) for line in file]
    return data

train_data = load_dataset('Datasets/train.jsonl')
validation_data = load_dataset('Datasets/validation.jsonl')
test_data = load_dataset('Datasets/test.jsonl')

In [6]:
def text_extract (data):
    text, label = [], []
    for idx in range(len(data)):
        text.append(data[idx]['text'])
        label.append(data[idx]['label'])
    return text, label

train_texts, train_labels= text_extract (train_data)

dev_texts, dev_labels= text_extract (validation_data)

test_texts, test_labels= text_extract (test_data)

In [7]:
# Load the pre-trained BERT tokenizer and model
#tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
tokenizer = ElectraTokenizer.from_pretrained('google/electra-base-discriminator')

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/27.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/666 [00:00<?, ?B/s]

In [8]:
# Tokenize the input texts
tokenized_train_texts = tokenizer(train_texts, padding=True, truncation=True, return_tensors='pt')

tokenized_dev_texts = tokenizer(dev_texts, padding=True, truncation=True, return_tensors='pt')

tokenized_test_texts = tokenizer(test_texts, padding=True, truncation=True, return_tensors='pt')

# Convert the labels to tensor
train_labels = torch.tensor(train_labels)

dev_labels = torch.tensor(dev_labels)

test_labels = torch.tensor(test_labels)

In [9]:
# Define hyperparameters
learning_rate = 8.1e-3
batch_size = 220
weight_decay = 2.5e-4
num_epochs = 100

In [10]:
# Create TensorDatasets for train, dev, and test sets
train_dataset = TensorDataset(tokenized_train_texts['input_ids'], tokenized_train_texts['attention_mask'], train_labels)

dev_dataset = TensorDataset(tokenized_dev_texts['input_ids'], tokenized_dev_texts['attention_mask'], dev_labels)

test_dataset = TensorDataset(tokenized_test_texts['input_ids'], tokenized_test_texts['attention_mask'], test_labels)


# Create DataLoaders for train, dev, and test sets
train_dataloader = DataLoader(train_dataset, batch_size= batch_size, shuffle= True)

dev_dataloader = DataLoader(dev_dataset, batch_size= batch_size, shuffle= False)

test_dataloader = DataLoader(test_dataset, batch_size= batch_size, shuffle= False)

In [11]:
# Load the pre-trained BERT model for sequence classification
#model = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels= 6)

model = ElectraForSequenceClassification.from_pretrained('google/electra-base-discriminator', num_labels=6)

# Freeze all layers except the classification layer
for param in model.parameters():
    param.requires_grad = False
for param in model.classifier.parameters():
    param.requires_grad = True

pytorch_model.bin:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of ElectraForSequenceClassification were not initialized from the model checkpoint at google/electra-base-discriminator and are newly initialized: ['classifier.out_proj.bias', 'classifier.dense.bias', 'classifier.out_proj.weight', 'classifier.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [12]:
# Define the optimizer for training the softmax layer
optimizer = optim.Adam(model.classifier.parameters(), lr= learning_rate, weight_decay= weight_decay)

device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
model.to(device)

# Define the loss function
criterion = nn.CrossEntropyLoss()

In [13]:
num_training_steps = num_epochs * len(train_dataloader)
lr_scheduler = get_scheduler(
    "linear",
    optimizer= optimizer,
    num_warmup_steps= 0,
    num_training_steps= num_training_steps,
)

progress_bar = tqdm(range(num_training_steps))

  0%|          | 0/7300 [00:00<?, ?it/s]

In [ ]:
# Train the model
best_dev_accuracy = 0.0
best_model_state_dict = None
Validation_results= []

for epoch in range(num_epochs):
    # Training loop
    model.train()
    for batch in train_dataloader:
        input_ids, attention_mask, batch_labels = batch

        input_ids= input_ids.to(device)
        attention_mask= attention_mask.to(device)
        batch_labels= batch_labels.to(device)

        optimizer.zero_grad()
        outputs = model(input_ids= input_ids, attention_mask= attention_mask, labels= batch_labels)
        loss = outputs.loss
        loss.backward()
        optimizer.step()
        #### to show progress_bar
        lr_scheduler.step()
        progress_bar.update(1)


    # Validation loop
    model.eval()
    dev_correct = 0
    total_dev = 0
    y_true = []
    y_pred = []
    loss_epoch= []
    with torch.no_grad():
        for batch in dev_dataloader:
            input_ids, attention_mask, batch_labels = batch

            input_ids= input_ids.to(device)
            attention_mask= attention_mask.to(device)
            batch_labels= batch_labels.to(device)

            outputs = model(input_ids=input_ids, attention_mask=attention_mask)
            logits = outputs.logits
            _, predicted = torch.max(logits, 1)

            # Append true labels and predicted labels for later use
            y_true.extend(batch_labels.tolist())
            y_pred.extend(predicted.tolist())

            # Calculate the loss
            loss = criterion(logits, batch_labels)
            loss_epoch.append(loss)

    # Calculate accuracy and F1 score
    f1 = f1_score(y_true, y_pred, average='weighted')
    accuracy = accuracy_score(y_true, y_pred)

    # Calculate the average loss
    loss_epoch_np = [tensor.cpu().detach().numpy() for tensor in loss_epoch]
    average_loss= np.mean(loss_epoch_np)
    print(f'epoch No. : {epoch}, Devset Accuracy : {round(accuracy,5)}, Devset f1_score : {round(f1,5)}, Average loss: {round(average_loss.tolist(),5)}')

    Validation_results.append([accuracy, f1, average_loss])

    if accuracy > best_dev_accuracy:
        best_dev_accuracy = accuracy
        # Save the best model (optional)
        best_model_state_dict = model.state_dict()

epoch No. : 0, Devset Accuracy : 0.396, Devset f1_score : 0.30591, Average loss: 1.56663
epoch No. : 1, Devset Accuracy : 0.4395, Devset f1_score : 0.33887, Average loss: 1.4989
epoch No. : 2, Devset Accuracy : 0.4325, Devset f1_score : 0.34059, Average loss: 1.49147
epoch No. : 3, Devset Accuracy : 0.4025, Devset f1_score : 0.31649, Average loss: 1.51131
epoch No. : 4, Devset Accuracy : 0.4365, Devset f1_score : 0.34542, Average loss: 1.47
epoch No. : 5, Devset Accuracy : 0.446, Devset f1_score : 0.35979, Average loss: 1.46857
epoch No. : 6, Devset Accuracy : 0.4045, Devset f1_score : 0.31773, Average loss: 1.5005
epoch No. : 7, Devset Accuracy : 0.4495, Devset f1_score : 0.3411, Average loss: 1.43585
epoch No. : 8, Devset Accuracy : 0.466, Devset f1_score : 0.36526, Average loss: 1.4359
epoch No. : 9, Devset Accuracy : 0.4665, Devset f1_score : 0.37249, Average loss: 1.43499
epoch No. : 10, Devset Accuracy : 0.447, Devset f1_score : 0.35129, Average loss: 1.45336
epoch No. : 11, Devs

In [1]:
# Saving Testset Results
data = {
    'Validation_results': Validation_results,
}
df = pd.DataFrame(data)
df.to_csv('Outputs/Model-4-3_Validation_results.csv', index= False)

NameError: name 'Validation_results' is not defined

In [2]:
# Load the best model state dict
if best_model_state_dict is not None:
    model.load_state_dict(best_model_state_dict)

    # Define the directory path to save the model
    save_path = 'Saved Models/Model-4-3.pth'

    # Save the model state dictionary and other relevant information
    torch.save({
        'model_state_dict': best_model_state_dict,
        'tokenizer': tokenizer
    }, save_path)

NameError: name 'best_model_state_dict' is not defined

In [ ]:
# Evaluate on the test set
model.eval()
y_true_test = []
y_pred_test = []
loss_epoch= []

with torch.no_grad():
    for batch in test_dataloader:
        input_ids, attention_mask, batch_labels = batch

        input_ids= input_ids.to(device)
        attention_mask= attention_mask.to(device)
        batch_labels= batch_labels.to(device)

        outputs = model(input_ids=input_ids, attention_mask=attention_mask)
        logits = outputs.logits
        _, predicted = torch.max(logits, 1)

        # Append true labels and predicted labels for later use
        y_true_test.extend(batch_labels.tolist())
        y_pred_test.extend(predicted.tolist())

        # Calculate the loss
        loss = criterion(logits, batch_labels)
        loss_epoch.append(loss)

# Calculate accuracy and F1 score for the test set
test_accuracy = accuracy_score(y_true_test, y_pred_test)
test_f1 = f1_score(y_true_test, y_pred_test, average='weighted')

# Calculate the average loss
loss_epoch_np = [tensor.cpu().detach().numpy() for tensor in loss_epoch]
average_loss= np.mean(loss_epoch_np)

print(f"Testset accuracy: {round(test_accuracy,5)} , Testset F1 score: {round(test_f1,5)}, Average loss: {round(average_loss.tolist(),5)}")
Test_results= [test_accuracy, test_f1, average_loss]

In [ ]:
# Saving Testset Results
data = {
    'Test_results': Test_results
}
df = pd.DataFrame(data)
df.to_csv('Outputs/Model-4-3_Test_results.csv', index= False)